# Matalb's code

```
function optimal_parameters = CGA(FitFun,initial_guess,lower_bound,uper_bound)
%Custom Genetic Algorithm. Developed by Edwin Ju�rez on 2016-03-21

% tic
% disp('********************************************************************************')
% fprintf('The current date and time is\n');
% disp(fix(clock))


%Initialize the parameters
parameters = initial_guess;

chromosome_length = length(parameters);
fitness = -inf;

% fprintf('The initial guess is:');
% parameters
fitness = FitFun(initial_guess)

%  Choose parameters:
%%% { Population Size, N : Depends on the dimensions of the sample space
PopSize = 2E2;

%%% { Number of mating individuals must be an even number
FittestNumber = min(PopSize*0.1,10^3*0.05); % 10 percent of the population will reproduce up to 50 individuals

%%% { Number of "Elite" individuals who will remain from Gen[i] to Gen[i+1]
Elite = min(PopSize*0.1,10^3*0.05); %10 percent of the population up to 50 individuals

%%% { Number of Generations to Simulate: How many iterations to simulate? Presumably the more the better.
LastGen = 75;

%%% { Mutation Rates: Probability of each gene (parameter) having a point mutation.
MutRate = 0.98; % there is a 98 percent chance of a mutation. There is a lot of genetic variation!
MutMagnitude = 2;%with a mutation the parameter will change up to 100%

%%% { Crossover Points: Location(s) where the individuals swap genes in producing children (next generation).
% CrossPoint = 1; %swap (after) the first "chromosome" [parameter]
CrossPoint = floor(1+chromosome_length*rand()); % Discrete Uniform RV ranging from 1 to chromosome_length (the number of parameters)

%  Initialize the G[0] population randomly: Create a set of N solutions randomly
% Gi = RandomParameters(PopSize,parameters);
Gi = RandomParameters(PopSize,parameters,1,lower_bound,uper_bound);
vanguardia=Gi(1:4,:);
prev_vanguardia = Gi(1,:);


% figure(1)
% set(1,'Units', 'Normalized', 'OuterPosition', [0 0 1 1]); 
% hold on

%This fileID will be used store the fittest individual of each Generation
% % % % fileID_fit = fopen(strcat(pwd,'\temp_outputs\',num2str(LastGen),'_Generations_Fittness_Evolution_',line,'.txt'),'at');
%This fileID will be used store the entire Generation!
% fileID_all = fopen(strcat(pwd,'\temp_outputs\',num2str(LastGen),'_Generations_All_Data_',line,'.txt'),'at');
% diary off
i=0;
fprintf('Runing the custom genetic algorithm\n')
while(i<LastGen)

%  Calculate fitness for population
% if(i==0) fprintf('Fitness\n'); toc; end
Gi_fit = GenerationFitness(FitFun,Gi);

%  Select mates to create children for the G1 (N+1) population
%%% { Mate selection: Individuals ranked proportional to their fittness
% if(i==0) fprintf('Rank\n'); end
% Gi_fit = RankGen(Gi_fit); %Order them from the most fit to the least fit
temp = sortrows([Gi Gi_fit],-(chromosome_length+1));
Gi_fittest = temp(1:FittestNumber,1:chromosome_length); %Consider only the fittest individuals
%%% { Randomly assign mates
Gi_mate = Gi_fittest(randperm(FittestNumber),:);
Gi_mate_1 = Gi_mate(1:FittestNumber/2,:);
Gi_mate_2 = Gi_mate(FittestNumber/2+1:end,:);

%%% { Mate: Genes are exchanged prescribed by cross-over points
% if(i==0) fprintf('Mate\n');  toc; end
Offsprings = crossover(Gi_mate_1,Gi_mate_2,CrossPoint);
% Offsprings = Gi_fittest; % Turning Crossover off!!
%%% { Introduce point mutations: 
% if(i==0) fprintf('Mutate\n');  toc; end
Offsprings = mutate(Offsprings,MutRate,MutMagnitude,i);
Offsprings = CheckBoundaries(Offsprings,lower_bound,uper_bound);
%%% { Clone the Elite members and mutate the clones
Clones = mutate(Gi_fittest(1:Elite,:),MutRate,MutMagnitude,1);
Clones = CheckBoundaries(Clones,lower_bound,uper_bound);
% Clones_2 = mutate(Gi_fittest(1:round(Elite/10),:),MutRate,MutMagnitude*0.1,1);
% Clones_2 = CheckBoundaries(Clones_2,lower_bound,uper_bound);
% Clones_3 = mutate(Gi_fittest(1:round(Elite/10),:),MutRate,MutMagnitude*0.01,1);
% Clones_3 = CheckBoundaries(Clones_3,lower_bound,uper_bound);

if not(isequal(Gi_fittest(1,:),prev_vanguardia))
    prev_vanguardia = Gi_fittest(1,:);
    vanguardia = round(LinearSearch3D(FitFun,Gi_fittest(1,:),lower_bound,uper_bound));
end

%%% "Elite" fittest individuals mate with the next generation,
%%% a mutated clone of some them also carries on.

%%% on each generation a number of random individuals show up equal to the
%%% number of Elite individuals
Gi = [vanguardia; Gi_fittest(1:Elite,:); Clones; Offsprings; RandomParameters(Elite,parameters,1,lower_bound,uper_bound)];
% Gi = [vanguardia; Gi_fittest(1:Elite,:); Clones; Clones_2; Clones_3; Offsprings; RandomParameters(Elite,parameters,1,lower_bound,uper_bound)];
% Gi = [Gi_fittest(1:Elite) Clones Offsprings];
%length of Gi = FittestNumber + 3*Elite;

%plot the Current Generation's fittest individuals
% plot(i+ones(1,length(Gi_fittest)),temp(1:FittestNumber,chromosome_length+1),'p')

%print to file the Current Generation's fittest individual

% % % % fprintf(fileID_fit,'%20.15f\t%20.15f\t%20.15f\t%20.15f\t%20.15f\t%20.15f',Dosage,Gi_fit(1).birth_rate,Gi_fit(1).death_rate,Gi_fit(1).clearance_rate,Gi_fit(1).fitness,i);
% % % % fprintf(fileID_fit,'\n');
% for j = 1:length(Gi)
%     fprintf(fileID_all,'%20.15f\t%20.15f\t%20.15f\t%20.15f\t%20.15f\t%20.15f\n',Dosage,Gi(j).birth_rate,Gi(j).death_rate,Gi(j).clearance_rate,Gi(j).fitness,i);
% end

fprintf('end of generation %d fittest value = %2.2g, params = [%d,%d,%d]\n',i,Gi_fit(1),Gi_fittest(1,1),Gi_fittest(1,2),Gi_fittest(1,3));
i = i+1;
%  Repeat fitness step and selection until the maximum number of generations is reached.
end
fprintf('After %d generations, the fittest value is %2.2g, params = [%d,%d,%d]\n',i,Gi_fit(1),Gi_fittest(1,1),Gi_fittest(1,2),Gi_fittest(1,3));
optimal_parameters = Gi_fittest(1,:);
end

function Parameters = RandomParameters(PopulationSize,OrigialParameters,scale,LowerBound,UpperBound)
%PopulationSize is the number of randomized sets of parameters this function generates.
%OriginalParemeters will be a first educated guess and the rest of the
%parameters will be generated around it.
%scale is the relative value of the change, scale = 1 means  new parameters
%will be roughly the same order of magnitude; scale = 0.1 means the new
%parameters will be roughly 1/10th of the original ones.


Parameters = zeros(PopulationSize,length(OrigialParameters));

for i = 1:PopulationSize
%     Parameters(i,:) = OrigialParameters;
    for j = 1:length(OrigialParameters);
        %NOTE: round() is used because all parameters must be integers in this example
        Parameters(i,j) = round(OrigialParameters(j)*(1+scale*(2*rand-1)));
        if Parameters(i,j) < LowerBound(j)
            Parameters(i,j) = LowerBound(j);
        elseif Parameters(i,j) > UpperBound(j)
            Parameters(i,j) = LowerBound(j);
        end
    end
end

end

function Gi_fit = GenerationFitness(FitFun,Generation)
%length of Gi - Generation i;
n = length(Generation);
Gi_fit = zeros(n,1);

for i = 1:n;
    %Compute fitness of specimen i
    Gi_fit(i) = FitFun(Generation(i,:));
    if isnan(Gi_fit(i))
        Gi_fit(i) = -inf;
    end
end
end


function Offspring = crossover(Gi_mate_1,Gi_mate_2,CrossPoint)

temp = size(Gi_mate_1);
n = temp(1);
number_of_chromosomes = temp(2);

for i = 1:n
    for j = 1:CrossPoint
        Offspring(i,j)   = Gi_mate_1(i,j);
        Offspring(n+i,j) = Gi_mate_2(i,j);
    end
    for j = CrossPoint+1:number_of_chromosomes
        Offspring(i,j)   = Gi_mate_2(i,j);
        Offspring(n+i,j) = Gi_mate_1(i,j);
    end
end

end

function G_mutated = mutate(Gi,MutRate,MutMagnitude,Mutation_dampering)

G_mutated = Gi;
temp = size(Gi);
n = temp(1);
number_of_chromosomes = temp(2);
decaying_rate = 0.9;

for i = 1:n
    for j = 1:number_of_chromosomes
        if ( binornd(1,MutRate) == 1 )
%             G_mutated(i).( Gfields{j} )   = Gi(i).( Gfields{j} )*(1+MutMagnitude*(2*rand()-1));
            %%Change the mutation here!!! = Original_value*(1+Maximum_Mutation_Percentage*Unif(-1,1)*decaying_rate^Mutation_dampering) 
            %NOTE: round() is used because all parameters must be integers in this example
            G_mutated(i,j) = round((Gi(i,j) + eps )*(1+MutMagnitude*(2*rand()-1))*decaying_rate^Mutation_dampering);
        else
            G_mutated(i,j) = Gi(i,j);
        end
    end
end

end

function Gi = CheckBoundaries(Gi,lower_bound,uper_bound)
temp = size(Gi);
n = temp(1);
number_of_chromosomes = temp(2);

for i = 1:n
    for j = 1:number_of_chromosomes
        if ( Gi(i,j)<lower_bound(j) )
            Gi(i,j) = lower_bound(j);
        end
        if( Gi(i,j)>uper_bound(j) )
            Gi(i,j) = uper_bound(j);
        end
    end
end

end

function vanguardia = LinearSearch3D(FitFun,Gi,lb,ub)

temp = size(Gi);
n = temp(1);
number_of_chromosomes = temp(2);

percent = 0.1;

vanguardia = zeros(n*4,number_of_chromosomes);

for i = 1:n
    % search the best x:
%     disp('searching x')
    vanguardia((i-1)*4+1,:) = LinearSearch(FitFun,Gi(i,:),1,lb*(1-percent),ub*(1+percent));
    % search the best y:
%     disp('searching y')
    vanguardia((i-1)*4+2,:) = LinearSearch(FitFun,Gi(i,:),2,lb*(1-percent),ub*(1+percent));
    % search the best z:
%     disp('searching z')
    vanguardia((i-1)*4+3,:) = LinearSearch(FitFun,Gi(i,:),3,lb*(1-percent),ub*(1+percent));
    % Do a local search on nearby
%     disp('searching local ball')
    vanguardia((i-1)*4+4,:) = LocalSearch(FitFun,Gi(i,:),percent*0.1,lb,ub);
%     disp('done searching local ball')
end

end

function max_x = LocalSearch(FitFun,initial_guess,percent,lb,ub)

lba = CheckBoundaries(round(initial_guess*(1-percent)),lb,ub);
uba = CheckBoundaries(round(initial_guess*(1+percent)),lb,ub);

counter = 1;
max_fit = -inf;
max_x = [0,0,0];
for a = lba(1):uba(1)
    for b = lba(2):uba(2)
        for c = lba(3):uba(3)
            temp_x = [a,b,c];
            temp_fit = FitFun(temp_x);
            if(temp_fit>max_fit)
                max_fit = temp_fit;
                max_x = temp_x;
            end
            counter = counter + 1;
        end
    end
end

end

function max_x = LinearSearch(FitFun,initial_guess,index,lb,ub)

% counter = 1;
max_fit = -inf;
max_x = initial_guess;
temp_x = initial_guess;
for variable_value = lb(index):ub(index)
    temp_x(index) = variable_value;
    temp_fit = FitFun(temp_x);
    if(temp_fit>max_fit)
        max_fit = temp_fit;
        max_x = temp_x;
    end
%     counter = counter + 1;
end

end
```

# Python Port

In [92]:
from operator import itemgetter

In [130]:
def enforce_bounds(params, LowerBound, UpperBound):
    new_params = []
    for i in np.arange(len(params)):
        currene_param = params[i]
        lb = LowerBound[i]
        ub = UpperBound[i]
        if currene_param<lb:
            currene_param = lb
        elif currene_param>ub:
            currene_param = ub
        new_params.append(currene_param)
    return new_params

In [131]:
def RandomParameters(PopulationSize,OrigialParameters,scale,LowerBound,UpperBound):
    #PopulationSize is the number of randomized sets of parameters this function generates.
    #OriginalParemeters will be a first educated guess and the rest of the
    #parameters will be generated around it.
    #scale is the relative value of the change, scale = 1 means  new parameters
    #will be roughly the same order of magnitude; scale = 0.1 means the new
    #parameters will be roughly 1/10th of the original ones.
    to_return = []
    i = 0
    while i<PopulationSize:
        temp = [(param+np.finfo(float).eps)*(1+scale*(2*np.random.uniform()-1)) for param in OrigialParameters]
        temp = enforce_bounds(temp, LowerBound, UpperBound)
        to_return.append(temp)
        
        i+=1
    return to_return

In [165]:
def GenerationFitness(FitFun,Generation):
    Gi_fit = []
    for i in np.arange(len(Generation)):
        Gi_fit.append(FitFun(Generation[i]))
        if np.isnan(Gi_fit[i]):
            Gi_fit[i] = numpy.NINF
    return Gi_fit

In [218]:
def crossover(Gi_mate_1,Gi_mate_2,CrossPoint):
    Offspring = []
    for parent1, parent2 in zip(Gi_mate_1,Gi_mate_2):
        parent1 = list(parent1)
        parent2 = list(parent2)
        off_1 = parent1[0:CrossPoint]+parent2[CrossPoint:]
        off_2 = parent2[0:CrossPoint]+parent1[CrossPoint:]
        Offspring.append(off_1)
        Offspring.append(off_2)
    return Offspring

In [264]:
def mutate(Offsprings,MutRate,MutMagnitude,Mutation_dampering,lb,ub):
    new_offsprings = []
    decaying_rate = 0.9
    
    # for each each offspring, chek if there will be a mutation on each gene
    for offspring in Offsprings:
        new_off = []
        for gene in offspring:
            if np.random.binomial(n=1,p=decaying_rate) == 1:
                new_gene = (gene+np.finfo(float).eps) * (1+ MutMagnitude*(2*np.random.uniform()-1)) * decaying_rate**Mutation_dampering
            else:
                new_gene = gene
            new_off.append(new_gene)
        new_offsprings.append(enforce_bounds(new_off,lb,ub))
    return new_offsprings

In [316]:
import numpy as np
def genetic_algorithm(FitFun, initial_guess, lower_bound, uper_bound, options=None):
    #Custom Genetic Algorithm. Developed by Edwin Juarez on 2016-03-21, ported to python on 2019-03-12

    #Initialize the parameters
    parameters = initial_guess
    chromosome_length = len(parameters)
    fitness = np.NINF

    # fprintf('The initial guess is:');
    # parameters
    fitness = FitFun(initial_guess)

    # Choose parameters:
    
    ## Population Size, N : Depends on the dimensions of the sample space
    PopSize = 2e2

    ## Number of mating individuals must be an even number
    FittestNumber = int(min(PopSize*0.1,10**3*0.05)) # 10 percent of the population will reproduce up to 50 individuals

    ## Number of "Elite" individuals who will remain from Gen[i] to Gen[i+1]
    Elite = int(min(PopSize*0.1,10**3*0.05)) # 10 percent of the population up to 50 individuals

    ## Number of Generations to Simulate: How many iterations to simulate? Presumably the more the better.
    LastGen = 75

    ## Mutation Rates: Probability of each gene (parameter) having a point mutation.
    MutRate = 0.98 # there is a 98 percent chance of a mutation. There is a lot of genetic variation!
    MutMagnitude = 2 # with a mutation the parameter will change up to 100%

    # Crossover Points: Location(s) where the individuals swap genes in producing children (next generation).
    # CrossPoint = 1; # swap (after) the first "chromosome" [parameter]
    CrossPoint = int(np.floor(1+chromosome_length*np.random.uniform())) # Discrete Uniform RV ranging from 1 to chromosome_length (the number of parameters)
    
    # Initialize the G[0] population randomly: Create a set of N solutions randomly
    # Gi = RandomParameters(PopSize,parameters);
    Gi = RandomParameters(PopSize,parameters,1,lower_bound,uper_bound);
#     vanguardia=Gi(1:4,:);
#     prev_vanguardia = Gi(1,:);
    # Running the main loop now
    i=0
    print('Runing the custom genetic algorithm')
#     print("The first generation takes the longest")
    while i<LastGen:
#         print(f"Generation {i}")
        # Calculate fitness for population
        Gi_fit = GenerationFitness(FitFun,Gi)

        ## Select mates to create children for the G1 (N+1) population
        ## Mate selection: Individuals ranked proportional to their fitness

        ###Order them from the most fit to the least fit
        temp = []
        for params, fitness in zip(Gi, Gi_fit):
            temp2 = params.copy()
            temp2.append(fitness)
            temp.append(temp2)
        temp = sorted(temp, key=itemgetter(len(temp[0])-1), reverse=True)
        ###Consider only the fittest individuals
        Gi_fittest = temp[0:FittestNumber]
        # Drop the fitness
        for x in Gi_fittest:
            del x[len(x)-1]

        ###Randomly assign mates
        Gi_mate = np.random.permutation(Gi_fittest) # permutate all individuals
        Gi_mate_1 = Gi_mate[0:int(FittestNumber/2)] # split the population in two
        Gi_mate_2 = Gi_mate[int(FittestNumber/2):]

        ### Mate: Genes are exchanged prescribed by cross-over points
        Offsprings = crossover(Gi_mate_1,Gi_mate_2,CrossPoint)

        ### Introduce point mutations: 
        Offsprings = mutate(Offsprings,MutRate,MutMagnitude,i,lower_bound,upper_bound)

        ### Clone the Elite members and mutate the clones
        Clones = mutate(Gi_fittest[0:Elite],MutRate,MutMagnitude,1,lower_bound,upper_bound)

        ### "Elite" fittest individuals mate with the next generation,
        ### a mutated clone of some them also carries on.

        ### on each generation a number of random individuals show up equal to the
        ### number of Elite individuals
        Gi = Gi_fittest[0:Elite]+Clones+Offsprings+RandomParameters(Elite,parameters,1,lower_bound,upper_bound)
        i += 1
    print(f'Done!, fittest individual was {Gi_fittest[0]} with fitness {FitFun(Gi_fittest[0])}')
    return Gi_fittest[0]

In [323]:
initial_guess = [0.1, 0.3, 3]
lower_bound = [0,0,0]
upper_bound = [5,5,10]
genetic_algorithm(FitFun, initial_guess, lower_bound, upper_bound, options=None)

Runing the custom genetic algorithm
Done!, fittest individual was [5, 5, 10] with fitness 20


[5, 5, 10]

In [ ]:
a

In [284]:
test_params = [0, 0.3, 3]
pop_size = 5
scale = 2
lower_bound = [0,0,0]
upper_bound = [5,5,10]
Gi = RandomParameters(pop_size,test_params,scale,lower_bound,upper_bound)
Gi

[[5.6566095390045385e-16, 0.11575848545590145, 0],
 [3.2014710224235903e-16, 0.6842835324411602, 5.856081820555885],
 [6.149177072925173e-17, 0.6018987264023944, 1.3416968439009427],
 [5.546123576816754e-16, 0.8910699677867546, 2.35598868282505],
 [4.413085076492548e-16, 0.10230225832585099, 8.01100125962347]]

In [285]:
Gi_fit = GenerationFitness(FitFun,Gi)
Gi_fit

[0.11575848545590202,
 6.540365352997045,
 1.9435955703033372,
 3.247058650611805,
 8.113303517949321]

NameError: name 'uper_bound' is not defined

In [286]:
parameters = test_params
FittestNumber = 4
chromosome_length = len(test_params)
CrossPoint = int(np.floor(1+chromosome_length*np.random.uniform()))
MutRate = 0.98 
MutMagnitude = 2
i =1
PopSize = 2e2
Elite = int(min(PopSize*0.1,10**3*0.05))

# Calculate fitness for population
Gi_fit = GenerationFitness(FitFun,Gi)

## Select mates to create children for the G1 (N+1) population
## Mate selection: Individuals ranked proportional to their fitness

###Order them from the most fit to the least fit
temp = []
for params, fitness in zip(Gi, Gi_fit):
    temp2 = params.copy()
    temp2.append(fitness)
    temp.append(temp2)
temp = sorted(temp, key=itemgetter(len(temp[0])-1), reverse=True)
###Consider only the fittest individuals
Gi_fittest = temp[0:FittestNumber]
# Drop the fitness
for x in Gi_fittest:
    del x[len(x)-1]
    
###Randomly assign mates
Gi_mate = np.random.permutation(Gi_fittest) # permutate all individuals
Gi_mate_1 = Gi_mate[0:int(FittestNumber/2)] # split the population in two
Gi_mate_2 = Gi_mate[int(FittestNumber/2):]

### Mate: Genes are exchanged prescribed by cross-over points
Offsprings = crossover(Gi_mate_1,Gi_mate_2,CrossPoint)

### Introduce point mutations: 
Offsprings = mutate(Offsprings,MutRate,MutMagnitude,i,lower_bound,upper_bound)

### Clone the Elite members and mutate the clones
Clones = mutate(Gi_fittest[0:Elite],MutRate,MutMagnitude,1,lower_bound,upper_bound)

### "Elite" fittest individuals mate with the next generation,
### a mutated clone of some them also carries on.

### on each generation a number of random individuals show up equal to the
### number of Elite individuals
Gi = Gi_fittest[0:Elite]+Clones+Offsprings+RandomParameters(Elite,parameters,1,lower_bound,upper_bound)

In [273]:
Gi

[[2.227710018905451e-16, 0.8454834390820937, 5.8139861010605385],
 [4.716768508240046e-16, 0.5321240983963764, 0],
 [5.926665112542162e-16, 0, 5.5886497367886765],
 [6.597767805618842e-16, 0.7999940895815129, 0],
 [1.3134052282032642e-17, 0.08431409594511605, 7.17373998212857]]

In [287]:
Gi

[[4.413085076492548e-16, 0.10230225832585099, 8.01100125962347],
 [3.2014710224235903e-16, 0.6842835324411602, 5.856081820555885],
 [5.546123576816754e-16, 0.8910699677867546, 2.35598868282505],
 [6.149177072925173e-17, 0.6018987264023944, 1.3416968439009427],
 [1.3445783187339322e-15, 0.12352088326685184, 0],
 [1.1879578117307199e-15, 0, 6.924237177854333],
 [7.233277492739289e-16, 1.9186245058660185, 5.063559700671208],
 [2.447938972723945e-16, 0.6463354532545422, 1.3416968439009427],
 [3.595653693898185e-16, 1.2509087829215695, 5.318431339839355],
 [4.174291025320603e-16, 0.6191135710423966, 0.46317511230415814],
 [1.2423851906332729e-16, 0, 8.01100125962347],
 [1.4431909927447866e-15, 0.23329562979652946, 0],
 [3.012327934213512e-16, 0.44258837247216665, 4.061007941427609],
 [3.352722641922906e-16, 0.3839124935542084, 3.752199705462107],
 [5.640980976827901e-17, 0.2693172092051552, 5.123236558959637],
 [4.3770525372372175e-16, 0.056109852646829654, 4.134995096009303],
 [2.866970879

In [235]:
Offsprings

[[1.3134052282032642e-17, 0.0, 5.5886497367886765],
 [5.926665112542162e-16, 0.08431409594511605, 7.17373998212857],
 [2.227710018905451e-16, 0.7999940895815129, 0.0],
 [6.597767805618842e-16, 0.8454834390820937, 5.8139861010605385]]

In [223]:
i

1

In [253]:
Offsprings

[[1.3134052282032642e-17, 0.0, 5.5886497367886765],
 [5.926665112542162e-16, 0.08431409594511605, 7.17373998212857],
 [2.227710018905451e-16, 0.7999940895815129, 0.0],
 [6.597767805618842e-16, 0.8454834390820937, 5.8139861010605385]]

In [263]:
mutate(Offsprings,MutRate,MutMagnitude,i,lower_bound,upper_bound)

[[1.2836452308289863e-17, 3.2726620640536846e-16, 0.48941575872711746],
 [8.264683071378796e-16, 0.08431409594511605, 9.218541667092806],
 [7.109100466192448e-16, 0, 3.3327759633231754e-16],
 [1.882103396815066e-15, 0, 10]]

[[4.3772617530906615e-16, 0, 1.3602157687904208],
 [0, 0.10383512756926427, 0],
 [5.420224068074272e-16, 0.021384059025442485, 3.6987346755535095],
 [6.462461031681022e-16, 0.43897650009117783, 0],
 [1.4565195441431915e-16, 0.8789684775564991, 0]]

[1.3602157687904213,
 0.10383512756926427,
 3.7201187345789526,
 0.4389765000911785,
 0.8789684775564992]

In [ ]:
function Gi_fit = GenerationFitness(FitFun,Generation)
%length of Gi - Generation i;
n = length(Generation);
Gi_fit = zeros(n,1);

for i = 1:n;
    %Compute fitness of specimen i
    Gi_fit(i) = FitFun(Generation(i,:));
    if isnan(Gi_fit(i))
        Gi_fit(i) = -inf;
    end
end
end

In [55]:
def FitFun(params):
    return sum(params)

In [ ]:

    
#     Gi_fittest = temp(1:FittestNumber,1:chromosome_length); #Consider only the fittest individuals
#     %%% { Randomly assign mates
#     Gi_mate = Gi_fittest(randperm(FittestNumber),:);
#     Gi_mate_1 = Gi_mate(1:FittestNumber/2,:);
#     Gi_mate_2 = Gi_mate(FittestNumber/2+1:end,:);

[[4.123066406888859e-16, 0.6217738835318609, 0.5888707931019983, 1.2106446766338597], [0, 0.4346074914415937, 7.0163685331558785, 7.450976024597472], [5.972084686675715e-16, 0, 8.095620233505377, 8.095620233505377], [0, 0.6694943001886696, 5.156810093897397, 5.826304394086066], [6.125350648193035e-16, 0.013484012306242842, 1.3976627157790973, 1.4111467280853407]]


In [101]:
sorted(temp, key=itemgetter(len(temp[0])-1), reverse=True)

[[5.972084686675715e-16, 0, 8.095620233505377, 8.095620233505377],
 [0, 0.4346074914415937, 7.0163685331558785, 7.450976024597472],
 [0, 0.6694943001886696, 5.156810093897397, 5.826304394086066],
 [6.125350648193035e-16,
  0.013484012306242842,
  1.3976627157790973,
  1.4111467280853407],
 [4.123066406888859e-16,
  0.6217738835318609,
  0.5888707931019983,
  1.2106446766338597]]

In [ ]:
# def rank_fitted_gen(gen):
#     # We know this is a list of list with the last parameter as the fitness
#     return 